In [2]:
from transformers import pipeline, AutoModel, AutoTokenizer, AutoModelForCausalLM
import torch
import os
import logging
import time
import pandas as pd

logging.basicConfig(level=logging.DEBUG)

MODELS_DIRECTORY = "D:/models/huggingface/"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"
DATASET_DIRECTORY = "D:/datasets/EmpiriNLP/FinQA/dataset"

num_outputs = 1

In [20]:
def load_dataset(dataset_directory: str, split: str = "train"):
    tsv_data = pd.read_csv(os.path.join(dataset_directory, f"{split}.tsv"), sep="\t")
    return tsv_data

def initialise_model(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # quantization_config = BitsAndBytesConfig(load_in_4bit=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        # quantization_config=quantization_config
    )

    return model, tokenizer

model, tokenizer = initialise_model(MODEL_ID)
dataset = load_dataset(DATASET_DIRECTORY)


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /meta-llama/Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 401 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json HTTP/11" 401 0
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /meta-llama/Llama-3.1-8B-Instruct/resolve/main/generation_config.json HTTP/11" 401 0


In [19]:
d = dataset.iloc[0]
q_cols = ["pre_text", "table", "post_text", "question"]
d = d[q_cols].copy()

s = "\n".join(d.values)
s

'interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) . if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million . foreign currency exposure as more fully described in note 2i . in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s . dollar-based exposures by entering into forward foreign currency exchange contracts . the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months . currently , our largest foreign currency exposure is the euro , primarily because our european operations have the highest proportion of our local currency denominated expenses . relative to foreign currency exposures existing at october 31 , 2009 and november 1 , 2008 , a 10% ( 10 % ) unfavorable movement in foreign cu

In [29]:
context

id                                      ADI/2009/page_49.pdf-1
pre_text     interest rate to a variable interest rate base...
table        |october 31 2009|november 1 2008\nfair value o...
post_text    fair value of forward exchange contracts after...
question             what is the the interest expense in 2009?
answer                                                     3.8
Name: 0, dtype: object

In [ ]:
context = dataset.iloc[0]
simple_query = "Who are you?"

q_cols = ["pre_text", "table", "post_text", "question"]
data = context[q_cols].copy()

template = [
    {"role": "system", "content": "You are a financial expert assistant that uses numerical reasoning to answer questions from snippets of text and tables. Provide strictly only the answer in one number or word."},
    {"role": "user", "content": simple_query}
]

# prompt = tokenizer.apply_chat_template(template, tokenize=True, add_generation_prompt=True)

# type(prompt[0])
tokenizer.pad_token = tokenizer.eos_token
tokenizer.apply_chat_template(template, tokenize=True, add_generation_prompt=True, return_tensors="pt", return_attention_mask=True, truncation=True, padding="max_length", max_length=2048).inputs_ids

AttributeError: 'list' object has no attribute 'inputs_ids'